# Projeto Titanic by Barbara Gomes - github @babitta 

#### Importando as bibliotecas

In [122]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier


#### Carregando os datasets 
Apos salvar os arquivos baixados do kaglle na pasta do projeto utilizaremos a função .read_csv para carregaras bases de dados.
Nomeamos as bases como train e teste.

In [123]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


Para visualizar os dados vamos usar a função .sample() que permitira verificar 5 linhas aleatorias da base de dados.

In [124]:
train.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
265,266,0,2,"Reeves, Mr. David",male,36.0,0,0,C.A. 17248,10.5000,NaN,S
393,394,1,1,"Newell, Miss. Marjorie",female,23.0,1,0,35273,113.2750,D36,C
53,54,1,2,"Faunthorpe, Mrs. Lizzie (Elizabeth Anne Wilkin...",female,29.0,1,0,2926,26.0000,NaN,S
164,165,0,3,"Panula, Master. Eino Viljami",male,1.0,4,1,3101295,39.6875,NaN,S
746,747,0,3,"Abbott, Mr. Rossmore Edward",male,16.0,1,1,C.A. 2673,20.2500,NaN,S


In [125]:
test.sample(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
159,1051,3,"Peacock, Mrs. Benjamin (Edith Nile)",female,26.0,0,2,SOTON/O.Q. 3101315,13.7750,NaN,S
392,1284,3,"Abbott, Master. Eugene Joseph",male,13.0,0,2,C.A. 2673,20.2500,NaN,S
350,1242,1,"Greenfield, Mrs. Leo David (Blanche Strouse)",female,45.0,0,1,PC 17759,63.3583,D10 D12,C
280,1172,3,"Oreskovic, Miss. Jelka",female,23.0,0,0,315085,8.6625,NaN,S
219,1111,3,"Thomson, Mr. Alexander Morrison",male,NaN,0,0,32302,8.0500,NaN,S


#### Seleção dos dados

Apos analise visual dos dados acima vamos retirar as colunas que consideramos desnessarias ao modelo e que não farão diferença na analise dos dados.
Neste caso avaliei as colunas de forma racional pensando o que poderia ou não influenciar no modelo,  mas é claro que dependo da analise é necessaria a avaliação de outros fatores como a correlação entre as variaveis.
Sendo assim as colunas a eliminar seraão: name, cabin e ticket.
usaremos a função .drop()
IMPORTANTE: a partir deste momento tudo que fizermos na base train deve ser replicado na base test , para que os resultados possam ser comparados ao final e comprovar a viabilidade do modelo.

In [87]:
train.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [88]:
#Importante verificar a base novamente para ver como ficou a apos a modificação. 
train.sample(5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,337,0,1,male,29.0,1,0,66.6000,S
594,595,0,2,male,37.0,1,0,26.0000,S
712,713,1,1,male,48.0,1,0,52.0000,S
82,83,1,3,female,NaN,0,0,7.7875,Q
851,852,0,3,male,74.0,0,0,7.7750,S


####  Tratamento de dados:
##### Valores Texto
Como nosso modelo de Machine Learning só trabalhara com  variaveis numericas, sera necessario tratar os dados relevantes
que estão como texto, as colunas Embarked e Sex.
Para isso vamos utilizar a função .get_dummies(), o objetivo desta função é codificar variáveis, criando novas colunas 
com valores numericos no dataset.
desta forma vamos criar tambem novos datasets para armazenar estas novas informações,vamos nomea-los como 
novo_train e novo_test.

In [89]:
novo_train = pd.get_dummies (train)
novo_test = pd.get_dummies (test)

In [90]:
#Verificar novamente como ficou o dataset apos as modificaçoes.
novo_train.sample(5)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
543,544,1,2,32.0,1,0,26.0000,0,1,0,0,1
523,524,1,1,44.0,0,1,57.9792,1,0,1,0,0
809,810,1,1,33.0,1,0,53.1000,1,0,0,0,1
558,559,1,1,39.0,1,1,79.6500,1,0,0,0,1


In [119]:
#Verificar novamente como ficou o dataset apos as modificaçoes.
novo_test.sample(5)

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
411,1303,1,37.0,1,0,90.0000,1,0,0,1,0
378,1270,1,55.0,0,0,50.0000,0,1,0,0,1
235,1127,3,20.0,0,0,7.8542,0,1,0,0,1
322,1214,2,26.0,0,0,13.0000,0,1,0,0,1
158,1050,1,42.0,0,0,26.5500,0,1,0,0,1


##### Valores Nulos

Da mesma forma que o modelo não aceita valores no formato texto o modelo tambem não aceita valores nulos e portando o devemos trata-los.
DETALHE IMPORTANTE: Como temos 2 bases esta etapa tem q ser realizada separadamente pois teremos tratativas diferentes a depender de quais campos estao nulos nas duas bases.

Vamos começar pela base train.

A primeira parte é identificar se existem valores nulos e onde estão localizados ( em quais colunas), vamos utilizar a função isnull() e a função sum() para quantificar os valores.

In [91]:
novo_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
Sex_female       0
Sex_male         0
Embarked_C       0
Embarked_Q       0
Embarked_S       0
dtype: int64

Como verificamos acima temos 177 valores nulos na coluna AGE (idade), sendo assim vamos substituir os nulos pela média de idade dos ocupantes, lembrando que cada coluna com valores nulos pede uma tratativa diferente devido ao tipo de dado e situação.

Utilizamos a formula fillna() para subistituição dos valores nulos e mean() para média 

In [92]:
novo_train["Age"].fillna(novo_train["Age"].mean(),inplace=True)


In [93]:
#vamos testar novamente para ver se deu certo 
novo_train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

Agora vamos ao dataset test
identificar se existem valores nulos e onde estão localizados 

In [94]:
novo_test.isnull().sum()

PassengerId     0
Pclass          0
Age            86
SibSp           0
Parch           0
Fare            1
Sex_female      0
Sex_male        0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

Observamos que na base test temos 86 valores nulos na coluna AGE e 1 valor nulo na coluna FARE, desta forma vamos replicar a formula para tratar a oluna age primeiro.

In [95]:
novo_test["Age"].fillna(novo_test["Age"].mean(),inplace=True)

In [96]:
#vamos testar  para ver se deu certo 
novo_test.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           1
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

Agora vamos tratar a coluna FARE, como esta coluna se trata de preço da passagem vamos imputar a média igual fizemos com a idade dos passageiros.

In [97]:
novo_test["Fare"].fillna(novo_test["Fare"].mean(),inplace=True)

In [98]:
#testar para ver se deu certo 
novo_test.isnull().sum()

PassengerId    0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
Sex_female     0
Sex_male       0
Embarked_C     0
Embarked_Q     0
Embarked_S     0
dtype: int64

Base de dados tratada , vamos criar o modelo.

#### Criando o Modelo de Machine Learning - Arvore de decisão

Nosso objetivo neste caso é prever a sobrevivencia dos passageiros, sendo assim vamos definir as nossas variaveis como:

Feature - dados de aprendizagem (passageiros), 

Target - o que queremos medir (sobrevivencia)


In [103]:
Feature = novo_train.drop('Survived',axis =1)
Target = novo_train['Survived']

In [104]:
arvore = DecisionTreeClassifier(max_depth=3, random_state =0)
arvore.fit(Feature,Target)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=3, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

In [105]:
arvore.score (Feature,Target)

0.8271604938271605

In [128]:
submission = pd.DataFrame()
submission['PassengerId'] = novo_test['PassengerId']
submission['Survived'] = arvore.predict(novo_test)

In [129]:
submission.to_csv('submission.csv', index=False)